In [ ]:
%pip install ray --quiet
%pip install lz4 

     |████████████████████████████████| 54.5 MB 1.3 MB/s 
     |████████████████████████████████| 8.8 MB 41.6 MB/s 
     |████████████████████████████████| 144 kB 69.4 MB/s 
     |████████████████████████████████| 4.1 MB 34.8 MB/s 
     |████████████████████████████████| 461 kB 59.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create a shortcut to shared folder and run this
%cd /content/drive/MyDrive/A3_PPO

/content/drive/MyDrive/A3_PPO


In [ ]:
import numpy as np
from tqdm import tqdm
import ray
import ray.rllib.agents.ppo as ppo
# from ray.tune.logger import pretty_print
import matplotlib.pyplot as plt
#
from main.continuous import FridayCleaning
from main.robot import Robot
from main.parser import parse_config, get_area

**Use this code in Chrome terminal (Ctrl+shift+i) to prevent session timeout :**

function ClickConnect(){
console.log("Working"); 
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
var clicker = setInterval(ClickConnect,60000);

**And the following code when you need to stop:**

clearInterval(clicker);

In [ ]:
# parent_path = Path(".").resolve().parent
grid = parse_config("grids/house.grid")
robot = Robot(init_position=(0, 8))
ray.shutdown()
ray.init()
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["num_workers"] = 2
config["env_config"] = {"robot": robot, "grid": grid}
config["recreate_failed_workers"] = True
config["gamma"] = 0.99
config["lr"] = 0.0001
config["clip_param"] = 0.2

PPO_trainer = ppo.PPOTrainer(env=FridayCleaning, config=config)

checkpoint_path = f"checkpoints/checkpoint{str(config['lr']).replace('.','')}_{str(config['gamma']).replace('.','')}"
train_losses = []
train_rewards = []
# trainer.train()
for i in tqdm(range(200)):
    # Perform one iteration of training the policy with PPO
    result = PPO_trainer.train()
    # print(result))
    train_rewards.append(result["episode_reward_mean"])
    train_losses.append(result["info"]["learner"]["default_policy"]["learner_stats"]["total_loss"])

checkpoint_path = PPO_trainer.save(checkpoint_dir=checkpoint_path)
print("checkpoint saved at", checkpoint_path)
plt.xlabel('train_epoch')
plt.ylabel('rewards')
plt.plot(train_rewards, label='Rewards')
plt.title(f"Rewards vs Epoch During Training (lr={config['lr']}, gamma={config['gamma']})")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

plt.xlabel('train_epoch')
plt.ylabel('losses')
plt.plot(train_losses, label='Losses')
plt.title(f"Losses vs Epoch During Training (lr={config['lr']}, gamma={config['gamma']})")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

np.savetxt(f"train_rewards{str(config['lr']).replace('.','')}_{str(config['gamma']).replace('.','')}.csv", 
           train_rewards,
           delimiter =", ", 
           fmt ='% s')

np.savetxt(f"train_losses{str(config['lr']).replace('.','')}_{str(config['gamma']).replace('.','')}.csv", 
           train_losses,
           delimiter =", ", 
           fmt ='% s')

#PPO_trainer.restore(f'{checkpoint_path}/checkpoint_000300/checkpoint-300')
PPO_trainer.restore(checkpoint_path)

test_cleaning = []
test_reward = []

for epoch in tqdm(range(10)):
    env = FridayCleaning(dict(robot=robot, grid=grid))
    obs = env.reset()
    #env.render()
    running_reward = 0.0
    initial_dust_area = sum([get_area(patch) for patch in env.grid.dirt_places]) + \
                        sum([get_area(patch) for patch in env.grid.much_dirt_places]) + sum([get_area(patch) for patch in env.grid.reg_dirt])
    s = 0
    done = False
    while not done:
        move = PPO_trainer.compute_action(obs)
        obs, reward, done, info = env.step(move)
        #env.render()
        #print(f"move: {move/(2*np.pi)*360}, reward: {reward}")
        running_reward += reward
        s += 1

    final_dust_area = sum([get_area(patch) for patch in env.grid.dirt_places]) + \
                      sum([get_area(patch) for patch in env.grid.much_dirt_places]) + sum([get_area(patch) for patch in env.grid.reg_dirt])

    cleaning_area = initial_dust_area - final_dust_area
    cleaning_per = (cleaning_area / initial_dust_area) * 100

    test_reward.append(running_reward)
    test_cleaning.append(cleaning_per)
    print(f"Epoch: {epoch} -- Reward: {running_reward} -- Cleaning %: {cleaning_per}")

print(test_reward)
print(test_cleaning)

2022-06-21 21:20:22,931	INFO logger.py:188 -- pip install "ray[tune]" to see TensorBoard files.
2022-06-21 21:20:22,933	WARNING logger.py:337 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2022-06-21 21:20:22,965	INFO trainer.py:2333 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-06-21 21:20:22,969	INFO ppo.py:415 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-06-21 21:20:22,970	INFO trainer.py:906 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=603) 2022-06-21 21:20:29,526	WARNING env.py:136 -- Your env doesn't have a .spec.max_episode_steps attribute. This